<a href="https://colab.research.google.com/github/Gaurav-30/aiLabPrograms/blob/main/Uncertain_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import time

class Parameter:
    variable_count = 1

    def __init__(self, name=None):
        if name:
            self.type = "Constant"
            self.name = name
        else:
            self.type = "Variable"
            self.name = "v" + str(Parameter.variable_count)
            Parameter.variable_count += 1

    def isConstant(self):
        return self.type == "Constant"

    def unify(self, type_, name):
        self.type = type_
        self.name = name

    def __eq__(self, other):
        return self.name == other.name

    def __str__(self):
        return self.name

class Predicate:
    def __init__(self, name, params):
        self.name = name
        self.params = params

    def __eq__(self, other):
        return self.name == other.name and all(a == b for a, b in zip(self.params, other.params))

    def __str__(self):
        return self.name + "(" + ",".join(str(x) for x in self.params) + ")"

    def getNegatedPredicate(self):
        return Predicate(negatePredicate(self.name), self.params)


class Sentence:
    sentence_count = 0

    def __init__(self, string):
        self.sentence_index = Sentence.sentence_count
        Sentence.sentence_count += 1
        self.predicates = []
        self.variable_map = {}
        local = {}

        for predicate in string.split("|"):
            name = predicate[:predicate.find("(")]
            params = []

            for param in predicate[predicate.find("(") + 1: predicate.find(")")].split(","):
                if param[0].islower():
                    if param not in local:  # Variable
                        local[param] = Parameter()
                        self.variable_map[local[param].name] = local[param]
                    new_param = local[param]
                else:
                    new_param = Parameter(param)
                    self.variable_map[param] = new_param

                params.append(new_param)

            self.predicates.append(Predicate(name, params))

    def getPredicates(self):
        return [predicate.name for predicate in self.predicates]

    def findPredicates(self, name):
        return [predicate for predicate in self.predicates if predicate.name == name]

    def removePredicate(self, predicate):
        self.predicates.remove(predicate)
        for key, val in self.variable_map.items():
            if not val:
                self.variable_map.pop(key)

    def containsVariable(self):
        return any(not param.isConstant() for param in self.variable_map.values())

    def __eq__(self, other):
        if len(self.predicates) == 1 and self.predicates[0] == other:
            return True
        return False

    def __str__(self):
        return "".join([str(predicate) for predicate in self.predicates])

class KB:
    def __init__(self, inputSentences):
        self.inputSentences = [x.replace(" ", "") for x in inputSentences]
        self.sentences = []
        self.sentence_map = {}

    def prepareKB(self):
        self.convertSentencesToCNF()
        for sentence_string in self.inputSentences:
            sentence = Sentence(sentence_string)
            for predicate in sentence.getPredicates():
                self.sentence_map[predicate] = self.sentence_map.get(predicate, []) + [sentence]

    def convertSentencesToCNF(self):
        for sentenceIdx in range(len(self.inputSentences)):
            if "=>" in self.inputSentences[sentenceIdx]:  # Do negation of the Premise and add them as literal
                self.inputSentences[sentenceIdx] = negateAntecedent(self.inputSentences[sentenceIdx])

    def askQueries(self, queryList):
        results = []

        for query in queryList:
            negatedQuery = Sentence(negatePredicate(query.replace(" ", "")))
            negatedPredicate = negatedQuery.predicates[0]
            prev_sentence_map = copy.deepcopy(self.sentence_map)
            self.sentence_map[negatedPredicate.name] = self.sentence_map.get(negatedPredicate.name, []) + [negatedQuery]
            self.timeLimit = time.time() + 40

            try:
                result = self.resolve([negatedPredicate], [False]*(len(self.inputSentences) + 1))
            except:
                result = False

            self.sentence_map = prev_sentence_map

            if result:
                results.append("TRUE")
            else:
                results.append("FALSE")

        return results

    def resolve(self, queryStack, visited, depth=0):
        if time.time() > self.timeLimit:
            raise Exception
        if queryStack:
            query = queryStack.pop(-1)
            negatedQuery = query.getNegatedPredicate()
            queryPredicateName = negatedQuery.name
            if queryPredicateName not in self.sentence_map:
                return False
            else:
                queryPredicate = negatedQuery
                for kb_sentence in self.sentence_map[queryPredicateName]:
                    if not visited[kb_sentence.sentence_index]:
                        for kbPredicate in kb_sentence.findPredicates(queryPredicateName):

                            canUnify, substitution = performUnification(copy.deepcopy(queryPredicate), copy.deepcopy(kbPredicate))

                            if canUnify:
                                newSentence = copy.deepcopy(kb_sentence)
                                newSentence.removePredicate(kbPredicate)
                                newQueryStack = copy.deepcopy(queryStack)

                                if substitution:
                                    for old, new in substitution.items():
                                        if old in newSentence.variable_map:
                                            parameter = newSentence.variable_map[old]
                                            newSentence.variable_map.pop(old)
                                            parameter.unify("Variable" if new[0].islower() else "Constant", new)
                                            newSentence.variable_map[new] = parameter

                                    for predicate in newQueryStack:
                                        for index, param in enumerate(predicate.params):
                                            if param.name in substitution:
                                                new = substitution[param.name]
                                                predicate.params[index].unify("Variable" if new[0].islower() else "Constant", new)

                                for predicate in newSentence.predicates:
                                    newQueryStack.append(predicate)

                                new_visited = copy.deepcopy(visited)
                                if kb_sentence.containsVariable() and len(kb_sentence.predicates) > 1:
                                    new_visited[kb_sentence.sentence_index] = True

                                if self.resolve(newQueryStack, new_visited, depth + 1):
                                    return True
                return False
        return True


def performUnification(queryPredicate, kbPredicate):
    substitution = {}
    if queryPredicate == kbPredicate:
        return True, {}
    else:
        for query, kb in zip(queryPredicate.params, kbPredicate.params):
            if query == kb:
                continue
            if kb.isConstant():
                if not query.isConstant():
                    if query.name not in substitution:
                        substitution[query.name] = kb.name
                    elif substitution[query.name] != kb.name:
                        return False, {}
                    query.unify("Constant", kb.name)
                else:
                    return False, {}
            else:
                if not query.isConstant():
                    if kb.name not in substitution:
                        substitution[kb.name] = query.name
                    elif substitution[kb.name] != query.name:
                        return False, {}
                    kb.unify("Variable", query.name)
                else:
                    if kb.name not in substitution:
                        substitution[kb.name] = query.name
                    elif substitution[kb.name] != query.name:
                        return False, {}
    return True, substitution


def negatePredicate(predicate):
    return predicate[1:] if predicate[0] == "~" else "~" + predicate


def negateAntecedent(sentence):
    antecedent = sentence[:sentence.find("=>")]
    premise = []

    for predicate in antecedent.split("&"):
        premise.append(negatePredicate(predicate))

    premise.append(sentence[sentence.find("=>") + 2:])
    return "|".join(premise)


def getInput(filename):
    with open(filename, "r") as file:
        noOfQueries = int(file.readline().strip())
        inputQueries = [file.readline().strip() for _ in range(noOfQueries)]
        noOfSentences = int(file.readline().strip())
        inputSentences = [file.readline().strip() for _ in range(noOfSentences)]
        return inputQueries, inputSentences


def printOutput(filename, results):
    print(results)
    with open(filename, "w") as file:
        for line in results:
            file.write(line)
            file.write("\n")
    file.close()


if __name__ == '__main__':
    inputQueries_, inputSentences_ = getInput("input.txt")
    knowledgeBase = KB(inputSentences_)
    knowledgeBase.prepareKB()
    results_ = knowledgeBase.askQueries(inputQueries_)
    printOutput("output.txt", results_)

['TRUE', 'TRUE']


In [4]:
pip install pomegranate

     |████████████████████████████████| 17.9MB 220kB/s 
     |████████████████████████████████| 15.3MB 321kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [1]:
import matplotlib.pyplot as plt 

import seaborn; seaborn.set_style('whitegrid') 

import numpy 

 

from pomegranate import * 

 

numpy.random.seed(0) 

numpy.set_printoptions(suppress=True) 

 

# The guests initial door selection is completely random 

guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3}) 

 

# The door the prize is behind is also completely random 

prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3}) 

 

    # Monty is dependent on both the guest and the prize.  

monty = ConditionalProbabilityTable( 

        [[ 'A', 'A', 'A', 0.0 ], 

         [ 'A', 'A', 'B', 0.5 ], 

         [ 'A', 'A', 'C', 0.5 ], 

         [ 'A', 'B', 'A', 0.0 ], 

         [ 'A', 'B', 'B', 0.0 ], 

         [ 'A', 'B', 'C', 1.0 ], 

         [ 'A', 'C', 'A', 0.0 ], 

         [ 'A', 'C', 'B', 1.0 ], 

         [ 'A', 'C', 'C', 0.0 ], 

         [ 'B', 'A', 'A', 0.0 ], 

         [ 'B', 'A', 'B', 0.0 ], 

         [ 'B', 'A', 'C', 1.0 ], 

         [ 'B', 'B', 'A', 0.5 ], 

         [ 'B', 'B', 'B', 0.0 ], 

         [ 'B', 'B', 'C', 0.5 ], 

         [ 'B', 'C', 'A', 1.0 ], 

         [ 'B', 'C', 'B', 0.0 ], 

         [ 'B', 'C', 'C', 0.0 ], 

         [ 'C', 'A', 'A', 0.0 ], 

         [ 'C', 'A', 'B', 1.0 ], 

         [ 'C', 'A', 'C', 0.0 ], 

         [ 'C', 'B', 'A', 1.0 ], 

         [ 'C', 'B', 'B', 0.0 ], 

         [ 'C', 'B', 'C', 0.0 ], 

         [ 'C', 'C', 'A', 0.5 ], 

         [ 'C', 'C', 'B', 0.5 ], 

         [ 'C', 'C', 'C', 0.0 ]], [guest, prize])   

 

         # State objects hold both the distribution, and a high level name. 

s1 = State(guest, name="guest") 

s2 = State(prize, name="prize") 

s3 = State(monty, name="monty") 

# Create the Bayesian network object with a useful name 

model = BayesianNetwork("Monty Hall Problem") 

 

# Add the three states to the network  

model.add_states(s1, s2, s3) 

# Add edges which represent conditional dependencies, where the second node is  

# conditionally dependent on the first node (Monty is dependent on both guest and prize) 

model.add_edge(s1, s3) 

model.add_edge(s2, s3) 

model.bake() 

model.probability([['A', 'B', 'C']]) 

model.probability([['A', 'B', 'C']]) 

print(model.predict_proba({})) 

print(model.predict_proba([[None, None, None]])) 

print(model.predict_proba([['A', None, None]])) 

print(model.predict_proba([{'guest': 'A', 'monty': 'B'}])) 

[{
     "class" : "Distribution",
     "dtype" : "str",
     "name" : "DiscreteDistribution",
     "parameters" : [
         {
             "A" : 0.33333333333333337,
             "B" : 0.33333333333333337,
             "C" : 0.33333333333333337
         }
     ],
     "frozen" : false
 }
 {
     "class" : "Distribution",
     "dtype" : "str",
     "name" : "DiscreteDistribution",
     "parameters" : [
         {
             "A" : 0.33333333333333337,
             "B" : 0.33333333333333337,
             "C" : 0.33333333333333337
         }
     ],
     "frozen" : false
 }
 {
     "class" : "Distribution",
     "dtype" : "str",
     "name" : "DiscreteDistribution",
     "parameters" : [
         {
             "B" : 0.3333333333333333,
             "A" : 0.3333333333333333,
             "C" : 0.3333333333333333
         }
     ],
     "frozen" : false
 }                                    ]
[array([{
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "